In [1]:
import xarray as xr
import numpy as np

def trim_to_PNW(data):
    """Trim data to Pacific Northwest region
    as defined in Bartusek et al. (2021)"""

    ## lon/lat range
    lat_range = [60, 40]
    lon_range = [230, 250]

    return data.sel(latitude=slice(*lat_range), longitude=slice(*lon_range))


def landarea_weighted_mean(data, lsm=None):
    """Get landarea-weighted mean on regular lon-lat grid.
    Specifically, weight by cosine of latitude"""

    ## get cos(lat)
    cos_lat = np.cos(np.deg2rad(data.latitude))

    ## multiply by fraction of land, if land-sea mask is provided
    if lsm is None:
        weights = cos_lat

    else:
        weights = cos_lat * lsm

    return data.weighted(weights=weights).mean(["latitude", "longitude"])

def preprocess(data, lsm):
    """pre-processing function to reduce data size"""

    ## trim LSM and data to Pac NW
    lsm_PNW = trim_to_PNW(lsm)
    data_PNW = trim_to_PNW(data)

    ## weighted mean
    data_PNW_mean = landarea_weighted_mean(data_PNW, lsm=lsm_PNW)

    return data_PNW_mean


In [5]:
## open dataset and select relevant variables
data = xr.open_zarr(
    "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr",
    chunks={"time":48},
)

lsm = data["land_sea_mask"]
data = data[["2m_temperature","2m_dewpoint_temperature","surface_pressure"]]

In [8]:
data_prepped = preprocess(data, lsm=lsm)

In [9]:
import time

In [ ]:
start = time.time()
data_prepped.load()
end = time.time()

In [ ]:
end

# scratch

In [14]:
data = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2/',
    # chunks={"time":24},
    consolidated=True,
)

## select relevant variables
# data = data[["t2m","d2m","sp",]]

# ## downsample in time
# data = data.isel(time=slice(None,None,4))
# data = data.sel(time=slice("1950","2023"))

In [26]:
data["tcc"].attrs["GRIB_name"]

'Total cloud cover'

In [17]:
list(data)

['cape',
 'd2m',
 'hcc',
 'istl1',
 'istl2',
 'istl3',
 'istl4',
 'lcc',
 'mcc',
 'msl',
 'p79.162',
 'p80.162',
 'siconc',
 'skt',
 'sp',
 'sst',
 'stl1',
 'stl2',
 'stl3',
 'stl4',
 'swvl1',
 'swvl2',
 'swvl3',
 'swvl4',
 't2m',
 'tcc',
 'tciw',
 'tclw',
 'tcrw',
 'tcsw',
 'tcw',
 'tcwv',
 'tsn',
 'u10',
 'u100',
 'v10',
 'v100',
 'z']

In [30]:
# data2 = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr")
data2 = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr")

In [39]:
data2[["2m_temperature","2m_dewpoint_temperature","surface_pressure","land_sea_mask"]]

<xarray.DataArray '2m_temperature' (time: 93544, latitude: 721, longitude: 1440)> Size: 388GB
dask.array<open_dataset-2m_temperature, shape=(93544, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 748kB 1959-01-01 ... 2023-01-10T18:00:00
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

In [35]:
list(data2)

['10m_u_component_of_wind',
 '10m_v_component_of_wind',
 '10m_wind_speed',
 '2m_dewpoint_temperature',
 '2m_temperature',
 'above_ground',
 'ageostrophic_wind_speed',
 'angle_of_sub_gridscale_orography',
 'anisotropy_of_sub_gridscale_orography',
 'boundary_layer_height',
 'divergence',
 'eddy_kinetic_energy',
 'geopotential',
 'geopotential_at_surface',
 'geostrophic_wind_speed',
 'high_vegetation_cover',
 'integrated_vapor_transport',
 'lake_cover',
 'land_sea_mask',
 'lapse_rate',
 'leaf_area_index_high_vegetation',
 'leaf_area_index_low_vegetation',
 'low_vegetation_cover',
 'mean_sea_level_pressure',
 'mean_surface_latent_heat_flux',
 'mean_surface_net_long_wave_radiation_flux',
 'mean_surface_net_short_wave_radiation_flux',
 'mean_surface_sensible_heat_flux',
 'mean_top_downward_short_wave_radiation_flux',
 'mean_top_net_long_wave_radiation_flux',
 'mean_top_net_short_wave_radiation_flux',
 'mean_vertically_integrated_moisture_divergence',
 'potential_vorticity',
 'relative_humidi

In [31]:
data2.time

<xarray.DataArray 'time' (time: 93544)> Size: 748kB
array(['1959-01-01T00:00:00.000000000', '1959-01-01T06:00:00.000000000',
       '1959-01-01T12:00:00.000000000', ..., '2023-01-10T06:00:00.000000000',
       '2023-01-10T12:00:00.000000000', '2023-01-10T18:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 748kB 1959-01-01 ... 2023-01-10T18:00:00

In [12]:
((data.latitude > 30) & (data.latitude<40)).sum().compute()

<xarray.DataArray 'latitude' ()> Size: 8B
array(38166)
Coordinates:
    depthBelowLandLayer  float64 8B 100.0
    entireAtmosphere     float64 8B 0.0
    number               int64 8B 0
    step                 timedelta64[ns] 8B 00:00:00
    surface              float64 8B 0.0

In [3]:
data["sp"]

<xarray.DataArray 'sp' (time: 1089864, values: 542080)> Size: 2TB
dask.array<open_dataset-sp, shape=(1089864, 542080), dtype=float32, chunksize=(24, 542080), chunktype=numpy.ndarray>
Coordinates:
    depthBelowLandLayer  float64 8B ...
    entireAtmosphere     float64 8B ...
    latitude             (values) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
    longitude            (values) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
    number               int64 8B ...
    step                 timedelta64[ns] 8B ...
    surface              float64 8B ...
  * time                 (time) datetime64[ns] 9MB 1900-01-01 ... 2024-04-30T...
    valid_time           (time) datetime64[ns] 9MB dask.array<chunksize=(24,), meta=np.ndarray>
Dimensions without coordinates: values
Attributes: (12/25)
    GRIB_N:                                   320
    GRIB_NV:                                  0
    GRIB_cfName:                              surface_air_pressure
    GRIB_cfVarName:                           sp
    GRIB_dataType:                            an
    GRIB_gridDefinitionDescription:           Gaussian Latitude/Longitude Grid
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_typeOfLevel:                         surface
    GRIB_units:                               Pa
    long_name:                                Surface pressure
    standard_name:                            surface_air_pressure
    units:                                    Pa

In [19]:
ml_surface[["t2m","d2m","sp",]].sel(time=slice("1950","2023")).isel(time=slice(None,None,4))

<xarray.Dataset> Size: 1TB
Dimensions:              (time: 162168, values: 542080)
Coordinates:
    depthBelowLandLayer  float64 8B ...
    entireAtmosphere     float64 8B ...
    latitude             (values) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
    longitude            (values) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
    number               int64 8B ...
    step                 timedelta64[ns] 8B ...
    surface              float64 8B ...
  * time                 (time) datetime64[ns] 1MB 1950-01-01 ... 2023-12-31T...
    valid_time           (time) datetime64[ns] 1MB dask.array<chunksize=(12,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    t2m                  (time, values) float32 352GB dask.array<chunksize=(12, 542080), meta=np.ndarray>
    d2m                  (time, values) float32 352GB dask.array<chunksize=(12, 542080), meta=np.ndarray>
    sp                   (time, values) float32 352GB dask.array<chunksize=(12, 542080), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              1
    GRIB_subCentre:            0
    history:                   2023-08-27T15:04 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  89c3caaaaf19eb0f6e9749dbde6c93972d64419137e2c8...
    pangeo-forge:recipe_hash:  09c22f5fffc2fbe3742fbb38c8c2c761e1b41b50570f09...
    pangeo-forge:version:      0.9.5.dev2+gd43015b